In [2]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objects as go

df = pd.read_excel('Data.xlsx', sheet_name=5)
ordered_df = df.iloc[::-1]

date_list = df.iloc[:, 0].values
price_list = df.iloc[:, 1].values
features_list = df.iloc[:, [1, 2, 3, 4]].values

#Scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features_list)
price_list = np.reshape(price_list, (price_list.shape[0], 1))
output_scaler = StandardScaler()
scaled_output_data = output_scaler.fit_transform(price_list)
print(scaled_output_data, scaled_data)
#Train test split
training_data_len = math.ceil(len(scaled_data)* .8)
training_data_len

#Train test split
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
look_back_window = 14

for i in range(look_back_window, len(train_data)):
    x_train.append(train_data[i-look_back_window:i])
    y_train.append(train_data[i, 0])
#Convert to np array
x_train, y_train = np.array(x_train), np.array(y_train)

test_data = scaled_data[training_data_len-look_back_window:, :]

x_test = []
y_test = price_list[training_data_len:]
y_test = np.array(y_test)
y_test = np.reshape(y_test, (y_test.shape[0], 1))

for i in range(look_back_window, len(test_data)):
    x_test.append(test_data[i-look_back_window:i])

x_test = np.array(x_test)

[[-0.48283747]
 [-0.4597184 ]
 [-0.48294487]
 ...
 [ 2.48179327]
 [ 2.46814442]
 [ 2.37603912]] [[-0.48283747  0.60379185 -0.46732082 -0.52125274]
 [-0.4597184   0.44544826 -0.47214525 -0.47219471]
 [-0.48294487 -0.04637153 -0.47917926 -0.46081404]
 ...
 [ 2.48179327 -0.57861472  2.53947557  2.55417938]
 [ 2.46814442 -0.91214917  2.47497296  2.52382099]
 [ 2.37603912 -0.8345886   2.43923359  2.41590326]]


In [3]:
from tensorflow import keras
from tensorflow.keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [13]:
input_shape = x_train.shape[1:]

head_size_list = [64, 128, 256, 512]
ff_dim_list = [1, 2, 4, 8, 16]
mlp_units_list = [64, 128, 256, 512]
mlp_dropout_list = [0.1, 0.2, 0.3, 0.4]
dropout_list = [0.1, 0.2, 0.3, 0.4]
result = []
for head_size in head_size_list:
    for ff_dim in ff_dim_list:
        for mlp_units in mlp_units_list:
            for mlp_dropout in mlp_dropout_list:
                for dropout in dropout_list:
                    model = build_model(
                        input_shape,
                        head_size=head_size,
                        num_heads=4,
                        ff_dim=ff_dim,
                        num_transformer_blocks=10,
                        mlp_units=[mlp_units],
                        mlp_dropout=mlp_dropout,
                        dropout=dropout,
                    )

                    model.compile(optimizer='adam', loss="mean_squared_error")

                    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

                    model.fit(
                        x_train,
                        y_train,
                        validation_split=0.2,
                        epochs=200,
                        batch_size=64,
                        callbacks=callbacks,
                        verbose=0
                    )
                    predictions = model.predict(x_test)
                    predictions = output_scaler.inverse_transform(predictions)
                    rmse = np.sqrt(np.mean(predictions - y_test)**2)
                    print(head_size, ff_dim, mlp_units, mlp_dropout, dropout)
                    print(rmse)
                    result.append({"rmse":rmse, "head_size":head_size, "ff_dim":ff_dim, "mlp_units":mlp_units, "mlp_dropout":mlp_dropout, "dropout":dropout})
                    if  rmse < 100:
                        train = np.squeeze(price_list[:training_data_len])
                        valid = np.squeeze(price_list[training_data_len:])
                        predictions = np.squeeze(predictions)
                        train_date_list = date_list[:training_data_len]
                        valid_date_list = date_list[training_data_len:]
                        valid_date_list2 = date_list[training_data_len:]
                        fig = go.Figure()
                        fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
                        fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
                        fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
                        fig.show()


8/8 [==============================] - 1s 27ms/step
64 1 64 0.1 0.1
2355.6368763497208
8/8 [==============================] - 1s 24ms/step
64 1 64 0.1 0.2
634.0488318962779
8/8 [==============================] - 1s 27ms/step
64 1 64 0.1 0.3
1921.9875372165588
8/8 [==============================] - 1s 29ms/step
64 1 64 0.1 0.4
1923.1459616560278
8/8 [==============================] - 1s 27ms/step
64 1 64 0.2 0.1
1907.3365158949443
8/8 [==============================] - 1s 26ms/step
64 1 64 0.2 0.2
1946.7698674892024
8/8 [==============================] - 2s 24ms/step
64 1 64 0.2 0.3
151.80522619728148
8/8 [==============================] - 1s 28ms/step
64 1 64 0.2 0.4
2380.936131472704
8/8 [==============================] - 1s 28ms/step
64 1 64 0.3 0.1
78.49964748475608


8/8 [==============================] - 1s 27ms/step
64 1 64 0.3 0.2
2060.3232449663365
8/8 [==============================] - 2s 28ms/step
64 1 64 0.3 0.3
2016.8379256581875
8/8 [==============================] - 1s 28ms/step
64 1 64 0.3 0.4
137.61263100228658
8/8 [==============================] - 2s 29ms/step
64 1 64 0.4 0.1
529.5040479706554
8/8 [==============================] - 1s 28ms/step
64 1 64 0.4 0.2
23.331681116615865


8/8 [==============================] - 1s 28ms/step
64 1 64 0.4 0.3
2099.541113678227
8/8 [==============================] - 1s 29ms/step
64 1 64 0.4 0.4
129.40358807323423
8/8 [==============================] - 1s 24ms/step
64 1 128 0.1 0.1
2251.667653947535
8/8 [==============================] - 1s 29ms/step
64 1 128 0.1 0.2
272.6424260829522
8/8 [==============================] - 1s 36ms/step
64 1 128 0.1 0.3
201.0854063452744
8/8 [==============================] - 1s 30ms/step
64 1 128 0.1 0.4
415.11871764640495
8/8 [==============================] - 1s 26ms/step
64 1 128 0.2 0.1
605.9082128509273
8/8 [==============================] - 1s 27ms/step
64 1 128 0.2 0.2
313.227109136814
8/8 [==============================] - 1s 24ms/step
64 1 128 0.2 0.3
233.88587199885671
8/8 [==============================] - 2s 36ms/step
64 1 128 0.2 0.4
28.886196924225104


8/8 [==============================] - 1s 29ms/step
64 1 128 0.3 0.1
2420.311676571234
8/8 [==============================] - 2s 39ms/step
64 1 128 0.3 0.2
117.41845981008636
8/8 [==============================] - 2s 31ms/step
64 1 128 0.3 0.3
356.3040491615854
8/8 [==============================] - 1s 27ms/step
64 1 128 0.3 0.4
2066.781215562278
8/8 [==============================] - 1s 25ms/step
64 1 128 0.4 0.1
1911.242677567645
8/8 [==============================] - 1s 28ms/step
64 1 128 0.4 0.2
331.0880422700711
8/8 [==============================] - 1s 29ms/step
64 1 128 0.4 0.3
395.660748539126
8/8 [==============================] - 1s 27ms/step
64 1 128 0.4 0.4
2320.0322710238825
8/8 [==============================] - 1s 29ms/step
64 1 256 0.1 0.1
477.70519205729164
8/8 [==============================] - 2s 26ms/step
64 1 256 0.1 0.2
694.8380690659933
8/8 [==============================] - 1s 26ms/step
64 1 256 0.1 0.3
502.61207920477636
8/8 [==============================] - 1

8/8 [==============================] - 1s 27ms/step
64 1 512 0.1 0.1
361.1847745569741
8/8 [==============================] - 1s 30ms/step
64 1 512 0.1 0.2
1453.8715158353975
8/8 [==============================] - 1s 27ms/step
64 1 512 0.1 0.3
2200.3780555290905
8/8 [==============================] - 1s 27ms/step
64 1 512 0.1 0.4
1780.339402907457
8/8 [==============================] - 1s 28ms/step
64 1 512 0.2 0.1
250.36747610200712
8/8 [==============================] - 1s 27ms/step
64 1 512 0.2 0.2
1959.1366262544464
8/8 [==============================] - 1s 27ms/step
64 1 512 0.2 0.3
132.42879708142783
8/8 [==============================] - 1s 27ms/step
64 1 512 0.2 0.4
594.683515148628
8/8 [==============================] - 1s 27ms/step
64 1 512 0.3 0.1
421.14066648405736
8/8 [==============================] - 1s 29ms/step
64 1 512 0.3 0.2
1556.7467500516073
8/8 [==============================] - 1s 27ms/step
64 1 512 0.3 0.3
269.3250512099848
8/8 [==============================] 

8/8 [==============================] - 1s 29ms/step
64 2 64 0.4 0.3
2228.028258582635
8/8 [==============================] - 1s 31ms/step
64 2 64 0.4 0.4
121.98598652661329
8/8 [==============================] - 1s 29ms/step
64 2 128 0.1 0.1
694.1433301972179
8/8 [==============================] - 1s 29ms/step
64 2 128 0.1 0.2
466.17634575076215
8/8 [==============================] - 1s 28ms/step
64 2 128 0.1 0.3
21.12706646182674


8/8 [==============================] - 1s 28ms/step
64 2 128 0.1 0.4
580.0592664268927
8/8 [==============================] - 1s 32ms/step
64 2 128 0.2 0.1
1814.278303738726
8/8 [==============================] - 1s 34ms/step
64 2 128 0.2 0.2
425.54973303321896
8/8 [==============================] - 1s 32ms/step
64 2 128 0.2 0.3
91.07566056910568


8/8 [==============================] - 2s 36ms/step
64 2 128 0.2 0.4
508.58035084794204
8/8 [==============================] - 1s 30ms/step
64 2 128 0.3 0.1
486.78514612709597
8/8 [==============================] - 1s 29ms/step
64 2 128 0.3 0.2
2194.6897943264103
8/8 [==============================] - 2s 38ms/step
64 2 128 0.3 0.3
727.8630508050685
8/8 [==============================] - 1s 32ms/step
64 2 128 0.3 0.4
1745.7999191160127
8/8 [==============================] - 1s 31ms/step
64 2 128 0.4 0.1
1945.4165568033857
8/8 [==============================] - 1s 28ms/step
64 2 128 0.4 0.2
1707.9614360033984
8/8 [==============================] - 1s 29ms/step
64 2 128 0.4 0.3
2307.3662360462718
8/8 [==============================] - 1s 29ms/step
64 2 128 0.4 0.4
126.94695618172004
8/8 [==============================] - 1s 29ms/step
64 2 256 0.1 0.1
1988.9700915825078
8/8 [==============================] - 1s 30ms/step
64 2 256 0.1 0.2
940.7750182609248
8/8 [=============================

8/8 [==============================] - 1s 31ms/step
64 2 256 0.3 0.1
610.8366012449186
8/8 [==============================] - 1s 28ms/step
64 2 256 0.3 0.2
771.8200115917173
8/8 [==============================] - 1s 29ms/step
64 2 256 0.3 0.3
743.947063960874
8/8 [==============================] - 1s 28ms/step
64 2 256 0.3 0.4
340.6901833635036
8/8 [==============================] - 2s 29ms/step
64 2 256 0.4 0.1
165.37511095496694
8/8 [==============================] - 1s 28ms/step
64 2 256 0.4 0.2
1933.2425644293066
8/8 [==============================] - 1s 30ms/step
64 2 256 0.4 0.3
2408.907612423781
8/8 [==============================] - 1s 31ms/step
64 2 256 0.4 0.4
601.9883802956681
8/8 [==============================] - 1s 29ms/step
64 2 512 0.1 0.1
1991.4107004057103
8/8 [==============================] - 2s 35ms/step
64 2 512 0.1 0.2
577.488065691692
8/8 [==============================] - 1s 32ms/step
64 2 512 0.1 0.3
854.0761409108231


KeyboardInterrupt: 

In [12]:
while True:
    model = build_model(
        input_shape,
        head_size=64,
        num_heads=4,
        ff_dim=1,
        num_transformer_blocks=10,
        mlp_units=[128],
        mlp_dropout=0.1,
        dropout=0.1,
    )

    model.compile(optimizer='adam', loss="mean_squared_error")

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    model.fit(
        x_train,
        y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=64,
        callbacks=callbacks,
    )
    predictions = model.predict(x_test)
    predictions = output_scaler.inverse_transform(predictions)
    rmse = np.sqrt(np.mean(predictions - y_test)**2)
    print(rmse)
    if  rmse < 100:
        break

Epoch 1/200
13/13 [==============================] - 13s 229ms/step - loss: 0.1114 - val_loss: 0.3203
Epoch 2/200
13/13 [==============================] - 2s 118ms/step - loss: 0.0666 - val_loss: 0.1728
Epoch 3/200
13/13 [==============================] - 2s 118ms/step - loss: 0.0380 - val_loss: 0.2061
Epoch 4/200
13/13 [==============================] - 2s 118ms/step - loss: 0.0277 - val_loss: 0.1162
Epoch 5/200
13/13 [==============================] - 2s 122ms/step - loss: 0.0179 - val_loss: 0.0584
Epoch 6/200
13/13 [==============================] - 2s 118ms/step - loss: 0.0174 - val_loss: 0.0446
Epoch 7/200
13/13 [==============================] - 2s 128ms/step - loss: 0.0179 - val_loss: 0.0655
Epoch 8/200
13/13 [==============================] - 1s 111ms/step - loss: 0.0162 - val_loss: 0.0375
Epoch 9/200
13/13 [==============================] - 1s 111ms/step - loss: 0.0243 - val_loss: 0.0416
Epoch 10/200
13/13 [==============================] - 2s 120ms/step - loss: 0.0182 - val_l

KeyboardInterrupt: 

In [62]:
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

8/8 [==============================] - 1s 85ms/step
1624.7013374142532


In [63]:
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()